# Backcasting Demo Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pickle
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from delphi.visualization import visualize
import delphi.jupyter_tools as jt
import numpy as np
import pandas as pd
from scipy import stats
#Comment out the next line if you do not have the delphi.db file. 
from delphi.db import engine
jt.print_commit_hash_message()
import random as rm
import delphi.evaluation as EN
import warnings
#warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from indra.statements import (
    Concept,
    Influence,
    Evidence,
    Event,
    QualitativeDelta,
)
from delphi.utils.indra import *
from delphi.utils.shell import cd
import seaborn as sns
import matplotlib.pyplot as plt
from delphi.utils.fp import flatMap, take, ltake, lmap, pairwise, iterate, exists
from delphi.cpp.AnalysisGraph import AnalysisGraph as AG

This notebook has been rendered with commit b'b0bba1c7d69871c07506a9abee9fa524d9d15123' of Delphi.


Here I will set random seeds

In [2]:
np.random.seed(87)
rm.seed(87)

In [3]:
statements = [ (("large", -1, "UN/entities/human/financial/economic/inflation"),("small", 1, "UN/events/human/human_migration"))]


Now we load the Causal Analysis Graph (CAG). This is CAG was inferred by reading in a JSON corpus and was pruned and adjusted to be human migration centered. Also is a list of the nodes contained in the CAG

In [4]:
#with open("../scripts/build/migration_centered_CAG.pkl",'rb') as f:
    #G = pickle.load(f)

G = AG.from_statements(statements)

In [5]:
G.map_concepts_to_indicators()


2

In [6]:
G.print_nodes()

In [7]:
n = 2
query = " ".join(
        [
            "select Source, Indicator from concept_to_indicator_mapping",
            "where `Concept` like 'UN/entities/human/financial/economic/inflation'",
        ]
    )

results = engine.execute(query)

pd.DataFrame(results, columns=results.keys())

,Source,Indicator
0,mitre12,Inflation Rate
1,mitre12,"Inflation, GDP deflator"
2,mitre12,"Inflation, GDP deflator: linked series"
3,mitre12,"Inflation, consumer prices"
4,mitre12,GDP growth


Next we map indicator variables to nodes. For the most part indicator variables can be inferred from available data and texts, but we can also manually map indicators to nodes. There is also a list of the indicator variables in the same order as the list of nodes above (i.e "Claims on other sectors of the domestic economy" is attached to "UN/events/human/economic_crisis".

In [8]:
query = " ".join(
        [
            f"select * from indicator",
            f"where `Variable` like 'Inflation Rate'",
        ]
    )

results = engine.execute(query)

list(results)

[(13606, 'South Sudan', None, 9.0, 'ieconomics.com', None, '%', '1.2', 'Inflation Rate', 2013),
 (13607, 'South Sudan', None, 10.0, 'ieconomics.com', None, '%', '-14.0', 'Inflation Rate', 2013),
 (13608, 'South Sudan', None, 11.0, 'ieconomics.com', None, '%', '-8.8', 'Inflation Rate', 2013),
 (13609, 'South Sudan', None, 12.0, 'ieconomics.com', None, '%', '16.0', 'Inflation Rate', 2013),
 (13610, 'South Sudan', None, 1.0, 'ieconomics.com', None, '%', '-2.9', 'Inflation Rate', 2014),
 (13611, 'South Sudan', None, 2.0, 'ieconomics.com', None, '%', '0.4', 'Inflation Rate', 2014),
 (13612, 'South Sudan', None, 3.0, 'ieconomics.com', None, '%', '-1.2', 'Inflation Rate', 2014),
 (13613, 'South Sudan', None, 4.0, 'ieconomics.com', None, '%', '-3.0', 'Inflation Rate', 2014),
 (13614, 'South Sudan', None, 5.0, 'ieconomics.com', None, '%', '0.6', 'Inflation Rate', 2014),
 (13615, 'South Sudan', None, 6.0, 'ieconomics.com', None, '%', '2.6', 'Inflation Rate', 2014),
 (13616, 'South Sudan', None, 

In [9]:
G.get_data_value_test("Inflation Rate",year=2013,month=9)

0.0